<a href="https://colab.research.google.com/github/ANUPRIYAR/Sentiment_Analysis_of_IMDB_MovieReviews/blob/main/SimpleAttentionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip

--2021-01-19 07:01:25--  https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84188 (82K) [application/x-httpd-php]
Saving to: ‘sentiment labelled sentences.zip’

sentiment labelled  100%[===================>]  82.21K  --.-KB/s    in 0.1s    

2021-01-19 07:01:26 (779 KB/s) - ‘sentiment labelled sentences.zip’ saved [84188/84188]



In [5]:
!unzip 'sentiment labelled sentences.zip'

Archive:  sentiment labelled sentences.zip
   creating: sentiment labelled sentences/
  inflating: sentiment labelled sentences/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/sentiment labelled sentences/
  inflating: __MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: sentiment labelled sentences/amazon_cells_labelled.txt  
  inflating: sentiment labelled sentences/imdb_labelled.txt  
  inflating: __MACOSX/sentiment labelled sentences/._imdb_labelled.txt  
  inflating: sentiment labelled sentences/readme.txt  
  inflating: __MACOSX/sentiment labelled sentences/._readme.txt  
  inflating: sentiment labelled sentences/yelp_labelled.txt  
  inflating: __MACOSX/._sentiment labelled sentences  


In [36]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split

In [9]:
df = pd.read_csv('/content/sentiment labelled sentences/amazon_cells_labelled.txt', delimiter='\t', names=['sentence', 'label'])

In [10]:
df.head(2)

,sentence,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1


In [15]:
df['word_count'] = df['sentence'].apply(lambda x: len(x.split()))

In [16]:
df.head(2)

,sentence,label,word_count
0,So there is no way for me to plug it in here i...,0,21
1,"Good case, Excellent value.",1,4


In [21]:
maxlen = np.max(df['word_count'])
maxlen

30

# **vocab_size**

In [23]:
words = set()
for sent in df['sentence']:
  for word in sent.split():
    words.add(word)
len(words)    

2815

In [24]:
vocab_size = len(words)

# **Preprocessing**

In [22]:
# Tokenize, convert to sequences and padding and calculate maximum length of sentence
sentences = df['sentence'].values
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
seq = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(seq, maxlen=maxlen, padding='post')

In [37]:
x= padded
y = df['label'].values

In [39]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((700, 30), (700,), (300, 30), (300,))

# **Now Let's define basic LSTM model**

In [25]:
padded.shape

(1000, 30)

In [26]:
features = padded.shape[1]

In [33]:
inputs1 = Input(shape=features,)
x1 = Embedding(input_dim=vocab_size+1, output_dim=maxlen, input_length=features)(inputs1)
x2 = LSTM(100, dropout=0.3)(x1)
output1 = Dense(1, activation='sigmoid')(x2)
model1 = Model(inputs1,output1)

In [34]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [35]:
model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 30, 30)            84480     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               52400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 136,981
Trainable params: 136,981
Non-trainable params: 0
_________________________________________________________________


In [40]:
model1.fit(x,y, validation_split=0.2, batch_size=64, epochs=10)

Epoch 1/10
13/13 [==============================] - 3s 87ms/step - loss: 0.6933 - acc: 0.5062 - val_loss: 0.6965 - val_acc: 0.4550
Epoch 2/10
13/13 [==============================] - 1s 45ms/step - loss: 0.6931 - acc: 0.5145 - val_loss: 0.6949 - val_acc: 0.4550
Epoch 3/10
13/13 [==============================] - 1s 44ms/step - loss: 0.6907 - acc: 0.5467 - val_loss: 0.6817 - val_acc: 0.5450
Epoch 4/10
13/13 [==============================] - 1s 45ms/step - loss: 0.5682 - acc: 0.7216 - val_loss: 0.5161 - val_acc: 0.7700
Epoch 5/10
13/13 [==============================] - 1s 45ms/step - loss: 0.2988 - acc: 0.8918 - val_loss: 0.5481 - val_acc: 0.7750
Epoch 6/10
13/13 [==============================] - 1s 45ms/step - loss: 0.1804 - acc: 0.9430 - val_loss: 0.4396 - val_acc: 0.7850
Epoch 7/10
13/13 [==============================] - 1s 46ms/step - loss: 0.1035 - acc: 0.9716 - val_loss: 0.7851 - val_acc: 0.7850
Epoch 8/10
13/13 [==============================] - 1s 46ms/step - loss: 0.0807 - a

In [47]:
inputs2 = Input(shape=features)
x1 = Embedding(input_dim=vocab_size+1, output_dim=maxlen, input_length=features)(inputs2)
x2 = LSTM(100, dropout=0.3, return_sequences=True)(x1)
output2 = Dense(1, activation='sigmoid', trainable=True)(x2)
model2 = Model(inputs2,output2)

In [48]:
model2.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 30, 30)            84480     
_________________________________________________________________
lstm_6 (LSTM)                (None, 30, 100)           52400     
_________________________________________________________________
dense_5 (Dense)              (None, 30, 1)             101       
Total params: 136,981
Trainable params: 136,981
Non-trainable params: 0
_________________________________________________________________


In [46]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model2.fit(x, y,batch_size=100,epochs=10,verbose=1,shuffle=True,validation_split=0.2)

Epoch 1/10
8/8 [==============================] - 3s 132ms/step - loss: 0.6932 - acc: 0.4811 - val_loss: 0.6964 - val_acc: 0.4603
Epoch 2/10
8/8 [==============================] - 1s 69ms/step - loss: 0.6910 - acc: 0.5350 - val_loss: 0.6996 - val_acc: 0.4630
Epoch 3/10
8/8 [==============================] - 1s 68ms/step - loss: 0.6894 - acc: 0.5648 - val_loss: 0.6921 - val_acc: 0.5362
Epoch 4/10
8/8 [==============================] - 1s 67ms/step - loss: 0.6745 - acc: 0.6628 - val_loss: 0.6466 - val_acc: 0.6752
Epoch 5/10
8/8 [==============================] - 1s 66ms/step - loss: 0.5436 - acc: 0.8079 - val_loss: 0.6284 - val_acc: 0.7433
Epoch 6/10
8/8 [==============================] - 1s 66ms/step - loss: 0.4215 - acc: 0.8714 - val_loss: 0.5631 - val_acc: 0.7627
Epoch 7/10
8/8 [==============================] - 1s 66ms/step - loss: 0.3635 - acc: 0.9157 - val_loss: 0.6307 - val_acc: 0.7615
Epoch 8/10
8/8 [==============================] - 1s 68ms/step - loss: 0.2881 - acc: 0.9342 - va